In [1]:
library(survival)
library(ggplot2)
library(dplyr)
library(reshape2)
library(stringr)

Warning message:
“package ‘survival’ was built under R version 4.0.5”
Warning message:
“package ‘ggplot2’ was built under R version 4.0.5”
Warning message:
“package ‘dplyr’ was built under R version 4.0.5”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘reshape2’ was built under R version 4.0.5”
Warning message:
“package ‘stringr’ was built under R version 4.0.5”


In [2]:
dir_fig = 'plots/'
file_dir = 'outputs/'

#use_gw_cov = FALSE
use_gw_cov = TRUE
gw_suf = ''
gw_var_base = ''
plat_var_suffs = c('_lr_il_ilgt', '_il')
if (use_gw_cov) {
    gw_suf = '_useGWB'
    gw_var_base = 'count_genomewide'
}

In [3]:
### clean features
fts = c('PLIp9_DEV_BR_GENE_10_cds + PLIp9_DEV_BR_GENE_10_utr', 
        'DEV_BR_GENE_10_cds + DEV_BR_GENE_10_utr', 
        'PLIp9_cds + PLIp9_utr', 
        'LOWPLI_cds + LOWPLI_utr', 
        'genic_cds + genic_utr',

        'PLIp9_DEV_BR_GENE_10_cds',
        'DEV_BR_GENE_10_cds',
        'PLIp9_cds',
        'LOWPLI_cds',
        'genic_cds',

        'PLIp9_DEV_BR_GENE_10_utr',
        'DEV_BR_GENE_10_utr',
        'PLIp9_utr',
        'LOWPLI_utr',
        'genic_utr',

        'PLIp9_DEV_BR_GENE_10_intron',
        'DEV_BR_GENE_10_intron', 
        'PLIp9_intron', 
        'LOWPLI_intron', 
        'genic_intron',
        
        'intergenic', 
        'FB_PR')
fts_labels = c('fetal brain expressed genes with pLI≥.9, exonic', 
               'fetal brain expressed genes, exonic', 
               'pLI≥.9, exonic', 
               'pLI≤.9, exonic', 
               'exonic',

               'fetal brain expressed genes with pLI≥.9, CDS', 
               'fetal brain expressed genes, CDS', 
               'pLI≥.9, CDS', 
               'pLI≤.9, CDS', 
               'CDS',

               'fetal brain expressed genes with pLI≥.9, UTR', 
               'fetal brain expressed genes, UTR', 
               'pLI≥.9, UTR', 
               'pLI≤.9, UTR', 
               'UTR',

               'fetal brain expressed genes with pLI≥.9, intronic',
               'fetal brain expressed genes, intronic', 
               'pLI≥.9, intronic', 
               'pLI≤.9, intronic', 
               'intronic', 
               
               'intergenic', 
               'fetal brain promoter')
fts_cats = c('fetal brain pLI≥.9', 
             'fetal brain',
             'pLI≥.9', 
             'pLI≤.9',
             'genomewide',
             
             'fetal brain pLI≥.9', 
             'fetal brain',
             'pLI≥.9', 
             'pLI≤.9',
             'genomewide',
             
             'fetal brain pLI≥.9', 
             'fetal brain',
             'pLI≥.9', 
             'pLI≤.9',
             'genomewide',
            
             'fetal brain pLI≥.9', 
             'fetal brain',
             'pLI≥.9', 
             'pLI≤.9',
             'genomewide', 
             
             'intergenic', 
             'fetal brain promoter')
fts_func = c('exonic', 
             'exonic',
             'exonic', 
             'exonic', 
             'exonic',
            
             'CDS', 
             'CDS',
             'CDS', 
             'CDS', 
             'CDS',
            
             'UTR', 
             'UTR',
             'UTR', 
             'UTR', 
             'UTR',
             
             'intronic', 
             'intronic',
             'intronic', 
             'intronic', 
             'intronic',
            
             'intergenic', 
             'fetal brain promoter')
fts_col_names = c('fetal_brain_genes_pli_exonic', 
                  'fetal_brain_genes_exonic', 
                  'pli_genes_exonic', 
                  'lowpli_genes_exonic', 
                  'genic_exonic',

                  'fetal_brain_genes_pli_CDS', 
                  'fetal_brain_genes_CDS', 
                  'pli_genes_CDS', 
                  'lowpli_genes_CDS', 
                  'genic_CDS',

                  'fetal_brain_genes_pli_UTR', 
                  'fetal_brain_genes_UTR', 
                  'pli_genes_UTR', 
                  'lowpli_genes_UTR', 
                  'genic_UTR',
                  
                  'fetal_brain_genes_pli_intronic', 
                  'fetal_brain_genes_intronic', 
                  'pli_genes_intronic', 
                  'lowpli_genes_intronic', 
                  'genic_intronic', 
                  
                  'intergenic', 
                  'FB_promoter')

In [4]:
get_pval_char <- function(pval) {
    if (pval>0.05) {return('')}
    if ((pval<0.05) && (pval>0.01)) {return('*')}
    else if ((pval<0.01) && (pval>0.001)) {return('**')}
    else {return('***')}
}

In [5]:
prs_mode = 'prscs'
file_suf = 'SQ20_SAMPLES_AD2_SAMPLES'
file_suf_trmissing = '25'
file_main = paste0(file_dir, 'combined_svs_trs_snvs_prs_pc_prscs_', file_suf, '_DELINS_GW_plat', '_TRmissingness', file_suf_trmissing, '_fltHomPol_famGT.csv')

df_main <- read.table(file_main, header = TRUE, sep = ",")
df_main[,'Platform'] <- as.factor(df_main[,'Platform'])
df_main[,'sex'] <- as.factor(df_main[,'sex'])
dim(df_main)
df_main

[1]  243 5574

fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,⋯,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q_del,count_ZS9_PLIp9_DEV_BR_GENE_10_intron_SUPP3_Q_ins,PRS_ASD_Z,PRS_SCZ_Z,PRS_EDU_Z,count_dnlof_inhlof,count_cnvs_sh,count_cnvs_cl,count_cnvs_all
<chr>,<chr>,<chr>,<fct>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>
F0001,REACH000001,ASD,Male,1,trio,1,1,0.008627126,-0.005607061,⋯,0,0,0,-0.971701072,-0.510485305,0.82162924,0,0,0,0
F0026,REACH000026,ASD,Male,1,trio,1,0,0.095363540,-0.045229620,⋯,0,0,0,0.303644763,0.817056455,0.06312390,1,0,0,0
F0058,REACH000058,ASD,Male,1,trio,1,1,0.002421759,-0.009216429,⋯,0,0,0,0.595643472,-0.305436874,1.94303491,2,0,0,0
F0065,REACH000065,ASD,Male,1,trio,1,0,0.252773800,0.107932900,⋯,0,0,0,1.166191828,0.943709479,-2.78896425,1,0,0,0
F0078,REACH000086,ASD,Male,1,trio,2,0,0.044785250,0.038064880,⋯,0,0,0,0.703902177,-0.152554399,0.06482359,0,0,0,0
F0078,REACH000089,ASD,Female,1,trio,2,0,0.049879690,0.047204300,⋯,0,0,0,0.384605619,0.209070495,0.16440487,0,0,0,0
F0081,REACH000092,ASD,Female,1,trio,1,1,-0.028510680,-0.001001894,⋯,0,0,0,-0.898010204,-0.994547648,0.53088874,1,0,0,0
F0081,REACH000095,CON,Female,1,trio,1,1,-0.030440730,-0.002736118,⋯,0,0,0,-1.036073363,-1.487948561,0.29231934,1,0,0,0
F0083,REACH000097,ASD,Male,1,trio,1,0,0.095793110,0.052887850,⋯,0,0,0,0.817923575,-0.003587407,-0.08546761,1,0,0,0


In [6]:
### correct SV counts of REACH000293
### add 1 count to the PLIp9_cds_lr_il_ilgt and DEV_BR_GENE_10_cds_lr_il_ilgt of REACH000293 because of his XYY genotype.
### XYY is considered high pli coding and fetal brain coding in the _lr_il_ilgt platform category.
### I counted 5 chrY genes in the fetal brain list (10), using gene symbols and getting coordinates by ensembl BioMart (Web Interface)
### NOTE: this correction is safe to be run only once to avoid over adding. If running twice is needed reinitiate the run from the beginning.
df_before = df_main[df_main$iid=='REACH000293',c('count_PLIp9_cds_lr_il_ilgt', 'count_DEV_BR_GENE_10_cds_lr_il_ilgt')]
print('df_before:')
df_before

[1] "df_before:"


,count_PLIp9_cds_lr_il_ilgt,count_DEV_BR_GENE_10_cds_lr_il_ilgt
,<int>,<int>
32,1,2


In [7]:
### correct SV counts of REACH000293
df_main[df_main$iid=='REACH000293',c('count_PLIp9_cds_lr_il_ilgt', 'count_DEV_BR_GENE_10_cds_lr_il_ilgt')] = df_before + 1
print('df_main:')
df_main[df_main$iid=='REACH000293',c('count_PLIp9_cds_lr_il_ilgt', 'count_DEV_BR_GENE_10_cds_lr_il_ilgt')]

[1] "df_main:"


,count_PLIp9_cds_lr_il_ilgt,count_DEV_BR_GENE_10_cds_lr_il_ilgt
,<dbl>,<dbl>
32,2,3


In [8]:
### ont/pb specific data frames
df_main_ont = df_main %>% filter(Platform == 'ONT')
df_main_pb = df_main %>% filter(Platform == 'PB')
print(dim(df_main_ont))
print(dim(df_main_pb))

[1]  101 5574
[1]  142 5574


In [9]:
### SVs All
### Notes: 
#     the log odds is based on natural logorithm. The wiskers of the coeficients are based on 95% confidence intervals
#     ANNOVA p-value computed based on LRT from null and the model is the same as the p-value reported in the model for the coefficient

#PRINT_INFO = TRUE
PRINT_INFO = FALSE
SHOW_FORMULA = TRUE

ORs = c()
ORs_plus = c()
ORs_minus = c()
log_ORs = c()
log_ORs_plus = c()
log_ORs_minus = c()
SEs = c()
pvals = c()
ano_pvals = c()
sum_cases = c()
sum_controls = c()
for (ft in fts) {

    ft_tmp = unlist(strsplit(ft, split='+', fixed=TRUE)) # split vars with +
    ft_tmp = str_trim(ft_tmp) # trim spaces
    ft_tmp = paste0('count_', ft_tmp) # add count_
    print(ft_tmp)
    ### make a new data frame and make a new variable called new_var
    new_df = df_main
    new_df['new_var'] = 0
    for (ft_ in ft_tmp) {
        for (plat_var_suff in plat_var_suffs) { # add burden detected from each platform
            ft_plat = paste0(ft_, plat_var_suff)
            new_df['new_var'] = new_df['new_var'] + df_main[ft_plat]
        }
    }

    var = 'new_var'
    ###print(var)

    ### make genomewide burden
    gw_var = ''
    if (use_gw_cov) {
        gw_var = 'gw_burden'
        gw_vs = paste0(gw_var_base, plat_var_suffs)
        new_df[gw_var] = 0
        for (gw_v in gw_vs) {
            new_df[gw_var] = new_df[gw_var] + new_df[gw_v]
        }
    }

    null_formula = as.formula(paste('Phenotype ~ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z +', gw_var,
                              '+ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + strata(fid)'))
    if (SHOW_FORMULA) {
        print(null_formula)
        SHOW_FORMULA = FALSE
    }
    
    sum_case = sum(new_df[new_df$Phenotype==1,][,var])
    sum_control = sum(new_df[new_df$Phenotype==0,][,var])
    sum_cases = c(sum_cases, sum_case)
    sum_controls = c(sum_controls, sum_control)
    
    model_formula = as.formula(paste('Phenotype ~', var, '+ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z +', gw_var,
                                     '+ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + strata(fid)'))
        
    null_model <- clogit(null_formula, data = new_df)
    #print(summary(null_model))
    
    clogit_model <- clogit(model_formula, data = new_df)
    #print(summary(clogit_model))
    #print(str(summary(clogit_model)))
    OR = summary(clogit_model)$conf.int[var, 'exp(coef)']
    OR_plus = summary(clogit_model)$conf.int[var, 'upper .95']
    OR_minus = summary(clogit_model)$conf.int[var, 'lower .95']
    log_OR = summary(clogit_model)$coefficients[var, 'coef']
    log_OR_plus = summary(clogit_model)$coefficients[var, 'coef'] + summary(clogit_model)$coefficients[var, 'se(coef)']
    log_OR_minus = summary(clogit_model)$coefficients[var, 'coef'] - summary(clogit_model)$coefficients[var, 'se(coef)']
    SE = summary(clogit_model)$coefficients[var, 'se(coef)']
    pval = summary(clogit_model)$coefficients[var, 'Pr(>|z|)']
    ORs = c(ORs, OR)
    ORs_plus = c(ORs_plus, OR_plus)
    ORs_minus = c(ORs_minus, OR_minus)
    log_ORs = c(log_ORs, log_OR)
    log_ORs_plus = c(log_ORs_plus, log_OR_plus)
    log_ORs_minus = c(log_ORs_minus, log_OR_minus)
    SEs = c(SEs, SE)
    pvals = c(pvals, pval)
        
    ano <- anova(null_model, clogit_model, test='LRT')
    ano_pval = ano$"Pr(>|Chi|)"[2]
    ano_pvals = c(ano_pvals, ano_pval)
}

sum_case_controls = paste(as.character(sum_cases), as.character(sum_controls), sep='/')

if (PRINT_INFO) {
    print('ORs:')
    print(ORs)
    print('ORs_plus:')
    print(ORs_plus)
    print('ORs_minus:')
    print(ORs_minus)
    print('log_ORs:')
    print(log_ORs)
    print('log_ORs_plus:')
    print(log_ORs_plus)
    print('log_ORs_minus:')
    print(log_ORs_minus)
    print('SEs:')
    print(SEs)
    print('pvals:')
    print(pvals)
    print('ano_pvals:')
    print(ano_pvals)
    print('sum_cases:')
    print(sum_cases)
    print('sum_controls:')
    print(sum_controls)
    print('sum_case_controls:')
    print(sum_case_controls)
}

my_data = data.frame('fts' = fts, 'fts_labels' = fts_labels, 'fts_cats' = fts_cats, 'fts_func' = fts_func,
                     'log_ORs' = log_ORs, 'log_ORs_plus' = log_ORs_plus, 'log_ORs_minus' = log_ORs_minus, 
                     'ano_pval' = ano_pvals, 'SEs' = SEs,
                     'sum_case_con' = sum_case_controls)

### write burden test tables
sub_fts_out = c('intergenic',
            'LOWPLI_intron', 'LOWPLI_cds + LOWPLI_utr', 
            'PLIp9_intron', 'PLIp9_cds + PLIp9_utr', 
            'DEV_BR_GENE_10_intron', 'DEV_BR_GENE_10_cds + DEV_BR_GENE_10_utr')
this_cols = c('fts_labels', 'log_ORs', 'log_ORs_plus', 'log_ORs_minus', 'ano_pval', 'SEs')
new_cols = c('Category', 'log OR', 'upper 95% CI', 'lower 95% CI', 'p-value', 'log OR SE')
this_data_out = my_data[my_data$fts %in% sub_fts_out, this_cols]
# formating the columns with number of digits
for (i_col in 2:length(this_cols)) {
    this_col = this_cols[i_col]
    this_data_out[this_col] = format(this_data_out[this_col], digits = 4, nsmall = 2)
}
colnames(this_data_out) <- new_cols
print(this_data_out)

[1] "count_PLIp9_DEV_BR_GENE_10_cds" "count_PLIp9_DEV_BR_GENE_10_utr"
Phenotype ~ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z + 
    gw_burden + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + 
    PC9 + PC10 + strata(fid)
[1] "count_DEV_BR_GENE_10_cds" "count_DEV_BR_GENE_10_utr"
[1] "count_PLIp9_cds" "count_PLIp9_utr"
[1] "count_LOWPLI_cds" "count_LOWPLI_utr"
[1] "count_genic_cds" "count_genic_utr"
[1] "count_PLIp9_DEV_BR_GENE_10_cds"
[1] "count_DEV_BR_GENE_10_cds"
[1] "count_PLIp9_cds"
[1] "count_LOWPLI_cds"
[1] "count_genic_cds"
[1] "count_PLIp9_DEV_BR_GENE_10_utr"
[1] "count_DEV_BR_GENE_10_utr"
[1] "count_PLIp9_utr"
[1] "count_LOWPLI_utr"
[1] "count_genic_utr"
[1] "count_PLIp9_DEV_BR_GENE_10_intron"
[1] "count_DEV_BR_GENE_10_intron"
[1] "count_PLIp9_intron"
[1] "count_LOWPLI_intron"
[1] "count_genic_intron"
[1] "count_intergenic"
[1] "count_FB_PR"
                                Category    log OR upper 95% CI lower 95% CI
2    fetal brain expressed genes, exonic  0.202359

In [10]:
### SVs ONT
#PRINT_INFO = TRUE
PRINT_INFO = FALSE
SHOW_FORMULA = TRUE

ORs = c()
ORs_plus = c()
ORs_minus = c()
log_ORs = c()
log_ORs_plus = c()
log_ORs_minus = c()
SEs = c()
pvals = c()
ano_pvals = c()
sum_cases = c()
sum_controls = c()
for (ft in fts) {

    ft_tmp = unlist(strsplit(ft, split='+', fixed=TRUE)) # split vars with +
    ft_tmp = str_trim(ft_tmp) # trim spaces
    ft_tmp = paste0('count_', ft_tmp) # add count_
    print(ft_tmp)
    ### make a new data frame and make a new variable called new_var
    new_df = df_main_ont
    new_df['new_var'] = 0
    for (ft_ in ft_tmp) {
        for (plat_var_suff in plat_var_suffs) { # add burden detected from each platform
            ft_plat = paste0(ft_, plat_var_suff)
            new_df['new_var'] = new_df['new_var'] + df_main_ont[ft_plat]
        }
    }

    var = 'new_var'
    ###print(var)
    
    ### make genomewide burden
    gw_var = ''
    if (use_gw_cov) {
        gw_var = 'gw_burden'
        gw_vs = paste0(gw_var_base, plat_var_suffs)
        new_df[gw_var] = 0
        for (gw_v in gw_vs) {
            new_df[gw_var] = new_df[gw_var] + new_df[gw_v]
        }
    }

    null_formula = as.formula(paste('Phenotype ~ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z +', gw_var,
                              '+ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + strata(fid)'))
    if (SHOW_FORMULA) {
        print(null_formula)
        SHOW_FORMULA = FALSE
    }
    
    sum_case = sum(new_df[new_df$Phenotype==1,][,var])
    sum_control = sum(new_df[new_df$Phenotype==0,][,var])
    sum_cases = c(sum_cases, sum_case)
    sum_controls = c(sum_controls, sum_control)
        
    model_formula = as.formula(paste('Phenotype ~', var, '+ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z +', gw_var,
                                     '+ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + strata(fid)'))
        
    null_model <- clogit(null_formula, data = new_df)
    #print(summary(null_model))
    
    clogit_model <- clogit(model_formula, data = new_df)
    #print(summary(clogit_model))
    #print(str(summary(clogit_model)))
    OR = summary(clogit_model)$conf.int[var, 'exp(coef)']
    OR_plus = summary(clogit_model)$conf.int[var, 'upper .95']
    OR_minus = summary(clogit_model)$conf.int[var, 'lower .95']
    log_OR = summary(clogit_model)$coefficients[var, 'coef']
    log_OR_plus = summary(clogit_model)$coefficients[var, 'coef'] + summary(clogit_model)$coefficients[var, 'se(coef)']
    log_OR_minus = summary(clogit_model)$coefficients[var, 'coef'] - summary(clogit_model)$coefficients[var, 'se(coef)']
    SE = summary(clogit_model)$coefficients[var, 'se(coef)']
    pval = summary(clogit_model)$coefficients[var, 'Pr(>|z|)']
    ORs = c(ORs, OR)
    ORs_plus = c(ORs_plus, OR_plus)
    ORs_minus = c(ORs_minus, OR_minus)
    log_ORs = c(log_ORs, log_OR)
    log_ORs_plus = c(log_ORs_plus, log_OR_plus)
    log_ORs_minus = c(log_ORs_minus, log_OR_minus)
    SEs = c(SEs, SE)
    pvals = c(pvals, pval)
        
    ano <- anova(null_model, clogit_model, test='LRT')
    ano_pval = ano$"Pr(>|Chi|)"[2]
    ano_pvals = c(ano_pvals, ano_pval)
}

sum_case_controls = paste(as.character(sum_cases), as.character(sum_controls), sep='/')

if (PRINT_INFO) {
    print('ORs:')
    print(ORs)
    print('ORs_plus:')
    print(ORs_plus)
    print('ORs_minus:')
    print(ORs_minus)
    print('log_ORs:')
    print(log_ORs)
    print('log_ORs_plus:')
    print(log_ORs_plus)
    print('log_ORs_minus:')
    print(log_ORs_minus)
    print('SEs:')
    print(SEs)
    print('pvals:')
    print(pvals)
    print('ano_pvals:')
    print(ano_pvals)
    print('sum_cases:')
    print(sum_cases)
    print('sum_controls:')
    print(sum_controls)
    print('sum_case_controls:')
    print(sum_case_controls)
}

my_data_all_ont = cbind(my_data, data.frame('log_ORs_ont' = log_ORs, 'log_ORs_plus_ont' = log_ORs_plus, 'log_ORs_minus_ont' = log_ORs_minus, 
                     'ano_pval_ont' = ano_pvals, 'SEs_ont' = SEs,
                     'sum_case_con_ont' = sum_case_controls))

[1] "count_PLIp9_DEV_BR_GENE_10_cds" "count_PLIp9_DEV_BR_GENE_10_utr"
Phenotype ~ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z + 
    gw_burden + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + 
    PC9 + PC10 + strata(fid)
[1] "count_DEV_BR_GENE_10_cds" "count_DEV_BR_GENE_10_utr"
[1] "count_PLIp9_cds" "count_PLIp9_utr"
[1] "count_LOWPLI_cds" "count_LOWPLI_utr"
[1] "count_genic_cds" "count_genic_utr"
[1] "count_PLIp9_DEV_BR_GENE_10_cds"
[1] "count_DEV_BR_GENE_10_cds"
[1] "count_PLIp9_cds"
[1] "count_LOWPLI_cds"
[1] "count_genic_cds"
[1] "count_PLIp9_DEV_BR_GENE_10_utr"
[1] "count_DEV_BR_GENE_10_utr"
[1] "count_PLIp9_utr"
[1] "count_LOWPLI_utr"
[1] "count_genic_utr"
[1] "count_PLIp9_DEV_BR_GENE_10_intron"
[1] "count_DEV_BR_GENE_10_intron"
[1] "count_PLIp9_intron"
[1] "count_LOWPLI_intron"
[1] "count_genic_intron"
[1] "count_intergenic"
[1] "count_FB_PR"


In [11]:
### SVs PB
#PRINT_INFO = TRUE
PRINT_INFO = FALSE
SHOW_FORMULA = TRUE

ORs = c()
ORs_plus = c()
ORs_minus = c()
log_ORs = c()
log_ORs_plus = c()
log_ORs_minus = c()
SEs = c()
pvals = c()
ano_pvals = c()
sum_cases = c()
sum_controls = c()
for (ft in fts) {

    ft_tmp = unlist(strsplit(ft, split='+', fixed=TRUE)) # split vars with +
    ft_tmp = str_trim(ft_tmp) # trim spaces
    ft_tmp = paste0('count_', ft_tmp) # add count_
    print(ft_tmp)
    ### make a new data frame and make a new variable called new_var
    new_df = df_main_pb
    new_df['new_var'] = 0
    for (ft_ in ft_tmp) {
        for (plat_var_suff in plat_var_suffs) { # add burden detected from each platform
            ft_plat = paste0(ft_, plat_var_suff)
            new_df['new_var'] = new_df['new_var'] + df_main_pb[ft_plat]
        }
    }

    var = 'new_var'
    ###print(var)

    ### make genomewide burden
    gw_var = ''
    if (use_gw_cov) {
        gw_var = 'gw_burden'
        gw_vs = paste0(gw_var_base, plat_var_suffs)
        new_df[gw_var] = 0
        for (gw_v in gw_vs) {
            new_df[gw_var] = new_df[gw_var] + new_df[gw_v]
        }
    }

    null_formula = as.formula(paste('Phenotype ~ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z +', gw_var,
                              '+ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + strata(fid)'))
    if (SHOW_FORMULA) {
        print(null_formula)
        SHOW_FORMULA = FALSE
    }
    
    sum_case = sum(new_df[new_df$Phenotype==1,][,var])
    sum_control = sum(new_df[new_df$Phenotype==0,][,var])
    sum_cases = c(sum_cases, sum_case)
    sum_controls = c(sum_controls, sum_control)
    
    model_formula = as.formula(paste('Phenotype ~', var, '+ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z +', gw_var,
                                     '+ PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + strata(fid)'))
        
    null_model <- clogit(null_formula, data = new_df)
    #print(summary(null_model))
    
    clogit_model <- clogit(model_formula, data = new_df)
    #print(summary(clogit_model))
    #print(str(summary(clogit_model)))
    OR = summary(clogit_model)$conf.int[var, 'exp(coef)']
    OR_plus = summary(clogit_model)$conf.int[var, 'upper .95']
    OR_minus = summary(clogit_model)$conf.int[var, 'lower .95']
    log_OR = summary(clogit_model)$coefficients[var, 'coef']
    log_OR_plus = summary(clogit_model)$coefficients[var, 'coef'] + summary(clogit_model)$coefficients[var, 'se(coef)']
    log_OR_minus = summary(clogit_model)$coefficients[var, 'coef'] - summary(clogit_model)$coefficients[var, 'se(coef)']
    SE = summary(clogit_model)$coefficients[var, 'se(coef)']
    pval = summary(clogit_model)$coefficients[var, 'Pr(>|z|)']
    ORs = c(ORs, OR)
    ORs_plus = c(ORs_plus, OR_plus)
    ORs_minus = c(ORs_minus, OR_minus)
    log_ORs = c(log_ORs, log_OR)
    log_ORs_plus = c(log_ORs_plus, log_OR_plus)
    log_ORs_minus = c(log_ORs_minus, log_OR_minus)
    SEs = c(SEs, SE)
    pvals = c(pvals, pval)
        
    ano <- anova(null_model, clogit_model, test='LRT')
    ano_pval = ano$"Pr(>|Chi|)"[2]
    ano_pvals = c(ano_pvals, ano_pval)
}

sum_case_controls = paste(as.character(sum_cases), as.character(sum_controls), sep='/')

if (PRINT_INFO) {
    print('ORs:')
    print(ORs)
    print('ORs_plus:')
    print(ORs_plus)
    print('ORs_minus:')
    print(ORs_minus)
    print('log_ORs:')
    print(log_ORs)
    print('log_ORs_plus:')
    print(log_ORs_plus)
    print('log_ORs_minus:')
    print(log_ORs_minus)
    print('SEs:')
    print(SEs)
    print('pvals:')
    print(pvals)
    print('ano_pvals:')
    print(ano_pvals)
    print('sum_cases:')
    print(sum_cases)
    print('sum_controls:')
    print(sum_controls)
    print('sum_case_controls:')
    print(sum_case_controls)
}

my_data_all_ont_pb = cbind(my_data_all_ont, data.frame('log_ORs_pb' = log_ORs, 'log_ORs_plus_pb' = log_ORs_plus, 'log_ORs_minus_pb' = log_ORs_minus, 
                     'ano_pval_pb' = ano_pvals, 'SEs_pb' = SEs,
                     'sum_case_con_pb' = sum_case_controls))

[1] "count_PLIp9_DEV_BR_GENE_10_cds" "count_PLIp9_DEV_BR_GENE_10_utr"
Phenotype ~ sex + MEAN_COVERAGE + count_dnlof_inhlof + PRS_ASD_Z + 
    gw_burden + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + 
    PC9 + PC10 + strata(fid)
[1] "count_DEV_BR_GENE_10_cds" "count_DEV_BR_GENE_10_utr"
[1] "count_PLIp9_cds" "count_PLIp9_utr"
[1] "count_LOWPLI_cds" "count_LOWPLI_utr"
[1] "count_genic_cds" "count_genic_utr"
[1] "count_PLIp9_DEV_BR_GENE_10_cds"
[1] "count_DEV_BR_GENE_10_cds"
[1] "count_PLIp9_cds"
[1] "count_LOWPLI_cds"
[1] "count_genic_cds"
[1] "count_PLIp9_DEV_BR_GENE_10_utr"
[1] "count_DEV_BR_GENE_10_utr"
[1] "count_PLIp9_utr"
[1] "count_LOWPLI_utr"
[1] "count_genic_utr"
[1] "count_PLIp9_DEV_BR_GENE_10_intron"
[1] "count_DEV_BR_GENE_10_intron"
[1] "count_PLIp9_intron"
[1] "count_LOWPLI_intron"
[1] "count_genic_intron"
[1] "count_intergenic"
[1] "count_FB_PR"


In [12]:
my_data_all_ont_pb['wt_ont'] = 1 / (my_data_all_ont_pb$SEs_ont^2)
my_data_all_ont_pb['wt_pb'] = 1 / (my_data_all_ont_pb$SEs_pb^2)
my_data_all_ont_pb['se_meta'] = sqrt(1 / (my_data_all_ont_pb['wt_ont'] + my_data_all_ont_pb['wt_pb']))
my_data_all_ont_pb['log_ORs_meta'] = ((my_data_all_ont_pb['log_ORs_ont']*my_data_all_ont_pb['wt_ont'] + 
                                          my_data_all_ont_pb['log_ORs_pb']*my_data_all_ont_pb['wt_pb']) / 
                                         (my_data_all_ont_pb['wt_ont'] + my_data_all_ont_pb['wt_pb']))
my_data_all_ont_pb['log_ORs_plus_meta'] = my_data_all_ont_pb['log_ORs_meta'] + my_data_all_ont_pb['se_meta']
my_data_all_ont_pb['log_ORs_minus_meta'] = my_data_all_ont_pb['log_ORs_meta'] - my_data_all_ont_pb['se_meta']
my_data_all_ont_pb['Z_meta'] = my_data_all_ont_pb['log_ORs_meta'] / my_data_all_ont_pb['se_meta']
my_data_all_ont_pb = my_data_all_ont_pb[!is.na(my_data_all_ont_pb['Z_meta']),]
my_data_all_ont_pb['ano_pval_meta'] = sapply(my_data_all_ont_pb['Z_meta'], function(x) 2 * (1 - pnorm(abs(x))))

In [13]:
### write burden test table
sub_fts_out = c('intergenic',
            'LOWPLI_intron', 'LOWPLI_cds + LOWPLI_utr', 
            'PLIp9_intron', 'PLIp9_cds + PLIp9_utr', 
            'DEV_BR_GENE_10_intron', 'DEV_BR_GENE_10_cds + DEV_BR_GENE_10_utr')
this_cols = c('fts_labels', 'log_ORs_meta', 'log_ORs_plus_meta', 'log_ORs_minus_meta', 'ano_pval_meta', 'se_meta')
new_cols = c('Category', 'log OR', 'upper 95% CI', 'lower 95% CI', 'p-value', 'log OR SE')
this_data_out = my_data_all_ont_pb[my_data_all_ont_pb$fts %in% sub_fts_out, this_cols]
# formating the columns with number of digits
for (i_col in 2:length(this_cols)) {
    this_col = this_cols[i_col]
    this_data_out[this_col] = format(this_data_out[this_col], digits = 4, nsmall = 2)
}
colnames(this_data_out) <- new_cols
print(this_data_out)

                                Category    log OR upper 95% CI lower 95% CI
2    fetal brain expressed genes, exonic  0.336878     0.520728    0.1530276
3                         pLI≥.9, exonic  0.309249     0.518287    0.1002120
4                         pLI≤.9, exonic  0.081523     0.163255   -0.0002088
17 fetal brain expressed genes, intronic -0.048814     0.009538   -0.1071674
18                      pLI≥.9, intronic -0.083555    -0.038322   -0.1287878
19                      pLI≤.9, intronic -0.021142     0.017336   -0.0596194
21                            intergenic  0.007904     0.040421   -0.0246126
   p-value log OR SE
2  0.06690   0.18385
3  0.13903   0.20904
4  0.31855   0.08173
17 0.40285   0.05835
18 0.06472   0.04523
19 0.58270   0.03848
21 0.80794   0.03252


[1] "plots/SVs_burden_SQ20_SAMPLES_AD2_SAMPLES_useGWB_META_main.png"


In [14]:
# plot SVs (meta), platform
sub_fts_labels = c('intergenic',
                   'pLI≤.9, intronic', 'pLI≤.9, exonic', 
                   'pLI≥.9, intronic', 'pLI≥.9, exonic', 
                   'fetal brain, intronic', 'fetal brain, exonic')
sub_fts = c('intergenic',
            'LOWPLI_intron', 'LOWPLI_cds + LOWPLI_utr', 
            'PLIp9_intron', 'PLIp9_cds + PLIp9_utr', 
            'DEV_BR_GENE_10_intron', 'DEV_BR_GENE_10_cds + DEV_BR_GENE_10_utr')
color_vec = c('black', 'green4', 'blue',  'green4', 'blue', 'green4', 'blue')
sub_fts_cats = c('intergenic', 'pLI≤.9', 'pLI≥.9', 'fetal brain')
sub_fts_func = c('exonic', 'intronic', 'intergenic')

fig_base = paste0('SVs_burden_', file_suf, gw_suf, '_META')
title = ''

my_data_sub = my_data_all_ont_pb[my_data_all_ont_pb$fts %in% sub_fts,]
my_data_sub$fts <- factor(my_data_sub$fts, levels = sub_fts)
my_data_sub$fts_cats <- factor(my_data_sub$fts_cats, levels = sub_fts_cats)
my_data_sub$fts_func <- factor(my_data_sub$fts_func, levels = sub_fts_func)
#my_data_sub['x_main'] = sub_fts_labels
#my_data_sub['fill_main'] = fill_main

my_data_sub.melt = melt(my_data_sub, id.vars = c('fts', 'fts_labels', 'fts_cats', 'fts_func',
                                                 #'x_main', 'fill_main',
                                                 # adding to bypass mega analysis
                                                 'log_ORs',
                                                 'log_ORs_plus', 'log_ORs_minus', 
                                                 'ano_pval', 
                                                 'sum_case_con', 
                                                 'log_ORs_plus_ont', 'log_ORs_minus_ont', 
                                                 'ano_pval_ont', 
                                                 'sum_case_con_ont',
                                                 'log_ORs_plus_pb', 'log_ORs_minus_pb', 
                                                 'ano_pval_pb', 
                                                 'sum_case_con_pb', 
                                                 'SEs', 'SEs_ont', 'SEs_pb',
                                                 'log_ORs_plus_meta', 'log_ORs_minus_meta', 
                                                 'wt_ont', 'wt_pb', 'se_meta', 'Z_meta', 'ano_pval_meta'), 
                        variable.name = 'lor_type', value.name = 'log_ORs_melt')

lor_type_levels = c('log_ORs_ont', 'log_ORs_pb', 'log_ORs_meta')
my_data_sub.melt$lor_type = factor(my_data_sub.melt$lor_type, levels = lor_type_levels)

my_data_sub.melt <- transform(my_data_sub.melt, log_ORs_plus = ifelse(lor_type=='log_ORs', log_ORs_plus,
                                                                             ifelse(lor_type=='log_ORs_ont', log_ORs_plus_ont, 
                                                                                           ifelse(lor_type=='log_ORs_pb', log_ORs_plus_pb, 
                                                                                                  ifelse(lor_type=='log_ORs_meta', log_ORs_plus_meta, '.')))))
my_data_sub.melt[,'log_ORs_plus'] <- as.double(my_data_sub.melt[,'log_ORs_plus'])

my_data_sub.melt <- transform(my_data_sub.melt, log_ORs_minus = ifelse(lor_type=='log_ORs', log_ORs_minus,
                                                                             ifelse(lor_type=='log_ORs_ont', log_ORs_minus_ont, 
                                                                                           ifelse(lor_type=='log_ORs_pb', log_ORs_minus_pb, 
                                                                                                  ifelse(lor_type=='log_ORs_meta', log_ORs_minus_meta, '.')))))
my_data_sub.melt[,'log_ORs_minus'] <- as.double(my_data_sub.melt[,'log_ORs_minus'])

my_data_sub.melt <- transform(my_data_sub.melt, ano_pval = ifelse(lor_type=='log_ORs', ano_pval,
                                                                             ifelse(lor_type=='log_ORs_ont', ano_pval_ont, 
                                                                                           ifelse(lor_type=='log_ORs_pb', ano_pval_pb, 
                                                                                                  ifelse(lor_type=='log_ORs_meta', ano_pval_meta, '.')))))
my_data_sub.melt[,'ano_pval'] <- as.double(my_data_sub.melt[,'ano_pval'])

sig <- sapply(my_data_sub.melt$ano_pval, get_pval_char)
my_data_sub.melt$sig = sig

#my_data_sub.melt

size_txt = 8
size_sig = 4
ylim_min = -0.1
ylim_max = 1.5

w1 = 0.7 # was 0.7
w2 = 0.2 # was 0.2

########################### main figure ###########################
fig_name = paste0(dir_fig, fig_base, "_main.png")

this_data = my_data_sub
sig <- sapply(this_data$ano_pval_meta, get_pval_char)
this_data$sig = sig

base <- ggplot(this_data, aes(x = fts_cats, y = log_ORs_meta, fill = fts_func, ymin=log_ORs_minus_meta , ymax=log_ORs_plus_meta)) +
  geom_bar(position = position_dodge(width=w1), stat = "identity", width = w1) +
  geom_errorbar(position = position_dodge(width=w1), aes(ymin = log_ORs_minus_meta, ymax = log_ORs_plus_meta), width = w2) +
  geom_hline(yintercept = 0, linetype = "dashed", color = "black") +
  geom_text(aes(label = sig, y = log_ORs_plus_meta+0.1), position=position_dodge(width=w1), angle=0, hjust=0.5, vjust=0.5, size=size_sig) +
  labs(y = "Log odds ratio", title = title) +
  theme(axis.text=element_text(size=size_txt),
        axis.text.x = element_text(angle = 15, vjust = 1, hjust=1., size=size_txt),
        axis.title.x = element_blank(),
        axis.title.y = element_text(size=size_txt),
        panel.background = element_blank(),
        plot.title = element_text(hjust = 0.5, size=size_txt), 
        legend.position = c(.15, 0.8),
        legend.text = element_text(size=size_txt-3),
        #legend.title = element_text(size=size_txt-5),
        legend.key.height = unit(0.1, "in"),         # Adjust legend key height
        legend.key.width = unit(0.1, "in")           # Adjust legend key width
        #plot.margin = margin(.20, 1.20, .20, .80, 'in') # Add margins: top, right, bottom, left (in points)
        ) +
  scale_fill_discrete(name = '')

print(fig_name)
print(base)
ggsave(fig_name, width = 2.5, height = 1.75, dpi = 1200, units = "in")